In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import scipy.io as sio
import numpy as np
import time 
import matplotlib.pyplot as plt

### setting parameters

In [ ]:
signal_model = 'tse'     #signal model to experiment on 

model_type     = 2
latent_nums    = [1,2,3,4]
learning_rates = [1e-4,1e-4,1e-4,1e-4]
all_epochs     = [200000,200000,200000,200000]

test_skip_idx = 10   #grab every 10th entry of dictionary for testing

### loading dictionary

In [ ]:
#Loading the baseline dictionary
dictionary = sio.loadmat('data/dictionary_%s.mat' % signal_model)['dictionary']
t2_range   = sio.loadmat('data/dict_params_%s.mat' % signal_model)['dict_params'][0][0][0]
    
t2_range_training = np.expand_dims(np.delete(t2_range,slice(None,None,test_skip_idx)),axis=0)
t2_range_testing  = t2_range[:,::test_skip_idx]

dictionary_training = np.delete(dictionary,slice(None,None,test_skip_idx),axis=1)
dictionary_testing  = dictionary[:,::test_skip_idx]
    
[T,Ntraining] = dictionary_training.shape
[_,Ntesting]  = dictionary_testing.shape

### defining auto-encoder 

In [ ]:
class autoencoder(nn.Module):
    def __init__(self,input_dimension,hidden_dimension):
        super().__init__()

        self.input_fc_1  = nn.Linear(input_dimension,input_dimension//2)
        self.input_fc_2  = nn.Linear(input_dimension//2,hidden_dimension)
        self.output_fc_1 = nn.Linear(hidden_dimension,input_dimension//2)
        self.output_fc_2 = nn.Linear(input_dimension//2,input_dimension)

    def forward(self,x):
        out = F.leaky_relu(self.input_fc_1(x))
        out = F.leaky_relu(self.input_fc_2(out))
        out = F.leaky_relu(self.output_fc_1(out))
        out = self.output_fc_2(out)
        return out

### training auto-encoder

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss()

#re-shaping dictionary, casting it to right type, and sending it to GPU
dictionary_for_training = torch.tensor(np.transpose(dictionary_training,(1,0)),dtype = torch.float).to(device)

models = []

for latent_num,epochs,learning_rate in zip(latent_nums,all_epochs,learning_rates):
    starting_time = time.time()
    
    print('Latent Variables: %d || epochs: %d || learning rate: %.6f' % (latent_num,epochs,learning_rate))
    model = autoencoder(T,latent_num)
    model.to(device)
    
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        output = model(dictionary_for_training)
        loss   = criterion(output,dictionary_for_training)
        loss.backward()
        optimizer.step()
        
        running_loss = loss.item()
        
        if(epoch % 1000 == 0):
            print('  current loss: %.12f' % running_loss)
    
    print('  final loss: %.12f' % running_loss)
    ending_time = time.time()
    print('  elapsed time: %.2f min' % ((ending_time - starting_time)/60))

    models.append(model)

### compute training and testing errors

In [ ]:
rmse = lambda truth, comp: np.sqrt(np.sum((comp - truth)**2,keepdims=False,axis=0)) / np.sqrt(np.sum(truth**2,keepdims=False,axis=0))

[u,s,v] = np.linalg.svd(dictionary,full_matrices=False)

all_basis = []

linear_errors_training   = []
proposed_errors_training = []

linear_errors_testing   = []
proposed_errors_testing = []

dictionary_linear_training   = []
dictionary_proposed_training = []

dictionary_linear_testing   = []
dictionary_proposed_testing = []

for latent_num, model in zip(latent_nums,models):
    #determine basis
    basis = u[:,0:latent_num]
    
    #estimate dictionaries with svd and autoencoder
    dictionary_estimate_linear_training   = basis @ np.conj(basis.T) @ dictionary_training
    dictionary_estimate_proposed_training = model(dictionary_for_training).cpu().detach().numpy().transpose((1,0))
    
    dictionary_estimate_linear_testing    = basis @ np.conj(basis.T) @ dictionary_testing
    dictionary_for_testing                = \
        torch.tensor(np.transpose(dictionary_testing,(1,0)),dtype = torch.float).to(device)
    dictionary_estimate_proposed_testing = \
        model(dictionary_for_testing).cpu().detach().numpy().transpose((1,0))

    error_linear_training   = rmse(dictionary_training,dictionary_estimate_linear_training)
    error_proposed_training = rmse(dictionary_training,dictionary_estimate_proposed_training)
    
    error_linear_testing    = rmse(dictionary_testing,dictionary_estimate_linear_testing)
    error_proposed_testing  = rmse(dictionary_testing,dictionary_estimate_proposed_testing)
    
    linear_errors_training.append(error_linear_training)
    proposed_errors_training.append(error_proposed_training)
    
    linear_errors_testing.append(error_linear_testing)
    proposed_errors_testing.append(error_proposed_testing)
    
    dictionary_linear_training.append(dictionary_estimate_linear_training)
    dictionary_proposed_training.append(dictionary_estimate_proposed_training)
    
    dictionary_linear_testing.append(dictionary_estimate_linear_testing)
    dictionary_proposed_testing.append(dictionary_estimate_proposed_testing)

### plot entry by entry training error

In [ ]:
for latent_num,error_linear,error_proposed in zip(latent_nums,linear_errors_training,proposed_errors_training):
    plt.title('Latent Variables: %d' % latent_num,fontsize=24)
    plt.plot(error_proposed)
    plt.plot(error_linear)
    plt.legend(['proposed','linear'],prop={'size':20})
    plt.show()

### plot entry by entry testing error

In [ ]:
lw = 3
for latent_num,error_linear,error_proposed in zip(latent_nums,linear_errors_testing,proposed_errors_testing):
    print('average percent errors:')
    print('  proposed: %.2f' % (np.mean(error_proposed)*100))
    print('  linear:   %.2f' % (np.mean(error_linear)*100))\
    
    plt.title('Latent Variables: %d' % latent_num,fontsize=24)
    plt.plot(error_proposed,linewidth = lw)
    plt.plot(error_linear,  linewidth = lw)
    plt.legend(['proposed','linear'],prop={'size':20})
    plt.show()